# AMBRS — Duncan Reproduction Notebook (`demo_duncan.ipynb`)

**Phases**
- **4a**: PartMC-CAMP vs MAM4 (native chemistry)
- **4b**: MAM4-CAMP vs MAM4 (native)
- **4c**: PartMC-CAMP vs MAM4-CAMP

All CAMP configuration (species, phases, mechanism) is generated **in this notebook** and written into each scenario directory.


In [1]:
from __future__ import annotations
import os, json, math
from pathlib import Path
import numpy as np

import ambrs
from ambrs import AerosolSpecies, GasSpecies, AerosolModalSizeDistribution, PoolRunner, Scenario
# from ambrs.visualization.output_pyparticle import plot_ensemble_variables

PARTMC_EXE = os.environ.get("PARTMC_EXE", "partmc")
MAM4_EXE   = os.environ.get("MAM4_EXE", "mam4")
ROOT = Path.cwd() / "runs_duncan"
PARTMC_DIR = ROOT / "partmc"
MAM4_DIR   = ROOT / "mam4"
for d in [ROOT, PARTMC_DIR, MAM4_DIR]: d.mkdir(parents=True, exist_ok=True)

RUN_4A = True
RUN_4B = False
RUN_4C = False


## Ensemble definition


In [ ]:
from ambrs.aerosol import AerosolSpecies, AerosolModalSizeDistribution
SO4 = AerosolSpecies("SO4", molar_mass=96.0)
POM = AerosolSpecies("POM", molar_mass=12.0)
SOA = AerosolSpecies("SOA", molar_mass=12.011)
BC  = AerosolSpecies("BC",  molar_mass=12.0)
DST = AerosolSpecies("DST", molar_mass=60.1)
NCL = AerosolSpecies("NCL", molar_mass=58.44)
MOM = AerosolSpecies("MOM", molar_mass=12.0)
aerosols = (SO4, POM, SOA, BC, DST, NCL, MOM)

SO2   = GasSpecies("SO2",   molar_mass=64.066)
H2O   = GasSpecies("H2O",   molar_mass=18.01528)
SOAG  = GasSpecies("SOAG",  molar_mass=12.011)
H2SO4 = GasSpecies("H2SO4", molar_mass=98.079)
gases = (SO2, H2O, SOAG, H2SO4)

size = AerosolModalSizeDistribution(number=(1.0e4,), geom_mean_diam=(1.1e-7,), geom_stdev=(1.6,))
N_MEMBERS = 3
dt, nsteps = 60.0, 60
gas_concs = (10e-9, 0.0, 5e-9, 0.0)
flux = 0.0
relative_humidity = 0.5
temperature = 298.0
pressure = 101325.0
height = 1000.0

scenarios = []
for i in range(N_MEMBERS):
    scenarios.append(Scenario(
        aerosols=aerosols, gases=gases, size=size.member(0), gas_concs=gas_concs,
        flux=flux, relative_humidity=relative_humidity, temperature=temperature, pressure=pressure, height=height,
    ))
scenario_names = [f"case_{i+1:02d}" for i in range(N_MEMBERS)]


TypeError: AerosolSpecies.__init__() missing 1 required positional argument: 'density'

## CAMP config writers (Duncan-style)


In [ ]:
def _solve_nstar_for_alpha(target_alpha: float, T: float, low=1.02, high=5.0, tol=1e-6, itmax=100):
    R = 8.314
    def alpha_of(N):
        N23 = N**(2.0/3.0)
        dH = 4.184e3 * (-10*(N-1.0) + 7.53*(N23-1.0) - 1.0)
        dS = 4.184e0 * (-32*(N-1.0) + 9.21*(N23-1.0) - 1.3)
        arg = (dH - T*dS)/(R*T)
        return 1.0/(1.0 + math.exp(arg))
    aL, aH = alpha_of(low), alpha_of(high)
    it = 0
    while (aL - target_alpha)*(aH - target_alpha) > 0 and it < 10:
        high *= 2.0
        aH = alpha_of(high)
        it += 1
    for _ in range(itmax):
        mid = 0.5*(low+high)
        am = alpha_of(mid)
        if abs(am - target_alpha) < tol:
            return mid
        if (aL - target_alpha)*(am - target_alpha) <= 0:
            high, aH = mid, am
        else:
            low, aL = mid, am
    return 0.5*(low+high)

def _simpol_B_params(p0_298_atm: float = 1e-10, dHv: float = 156e3, Tref: float = 298.0):
    # SIMPOL: log10 p_sat = B1*(1/T) + B2 (when B3=B4=0)
    # B1 chosen from enthalpy; B2 from p0 at Tref => B2 = log10(p0) - B1*(1/Tref)
    R = 8.314
    log10e = math.log10(math.e)
    B1 = -dHv * log10e / R
    B2 = math.log10(p0_298_atm) - B1*(1.0/Tref)  # correct sign
    return (B1, B2, 0.0, 0.0)

def _gas_diffusion_coeff(T: float, P: float):
    return 0.557e-4 * (T**1.75) / max(P, 1.0)

def write_duncan_camp_files(run_dir: Path, scenario: Scenario, mech_name: str = "MAM4_SOA_partitioning") -> Path:
    campdir = Path(run_dir)/"camp_config"
    campdir.mkdir(parents=True, exist_ok=True)

    phases = {"aerosol_phases": [{"name": "mixed", "type": "AERO_PHASE", "composition": ["SO4","POM","SOA","BC","DST","NCL","MOM"]}]}
    rep = {"aerosol_representation": {"name": "single", "type": "AERO_REP_SINGLE_PARTICLE", "n layers": 1, "layers": [{"name": "L1","phases": ["mixed"], "maximum computational particles": 1100}]}}

    T = float(scenario.temperature)
    P = float(scenario.pressure)
    D = _gas_diffusion_coeff(T, P)
    species = {
        "species": [
            {"name": "SO2",   "type": "CHEM_SPEC", "phase": "gas",  "properties": {"molecular weight [kg mol-1]": 0.064066}},
            {"name": "H2O",   "type": "CHEM_SPEC", "phase": "gas",  "properties": {"molecular weight [kg mol-1]": 0.01801528, "is gas-phase water": True}},
            {"name": "SOAG",  "type": "CHEM_SPEC", "phase": "gas",  "properties": {"molecular weight [kg mol-1]": 0.012011, "diffusion coeff [m2 s-1]": 0.81*D}},
            {"name": "H2SO4", "type": "CHEM_SPEC", "phase": "gas",  "properties": {"molecular weight [kg mol-1]": 0.098079, "diffusion coeff [m2 s-1]": D}},
            {"name": "SO4",   "type": "CHEM_SPEC", "phase": "mixed","properties": {"molecular weight [kg mol-1]": 0.096,    "density [kg m-3]": 1770}},
            {"name": "POM",   "type": "CHEM_SPEC", "phase": "mixed","properties": {"molecular weight [kg mol-1]": 0.012,    "density [kg m-3]": 1200}},
            {"name": "SOA",   "type": "CHEM_SPEC", "phase": "mixed","properties": {"molecular weight [kg mol-1]": 0.012011, "density [kg m-3]": 1200}},
            {"name": "BC",    "type": "CHEM_SPEC", "phase": "mixed","properties": {"molecular weight [kg mol-1]": 0.012,    "density [kg m-3]": 1500}},
            {"name": "DST",   "type": "CHEM_SPEC", "phase": "mixed","properties": {"molecular weight [kg mol-1]": 0.0601,   "density [kg m-3]": 2650}},
            {"name": "NCL",   "type": "CHEM_SPEC", "phase": "mixed","properties": {"molecular weight [kg mol-1]": 0.05844,  "density [kg m-3]": 2160}},
            {"name": "MOM",   "type": "CHEM_SPEC", "phase": "mixed","properties": {"molecular weight [kg mol-1]": 0.012,    "density [kg m-3]": 1200}},
        ]
    }

    B1, B2, B3, B4 = _simpol_B_params(p0_298_atm=1e-10, dHv=156e3, Tref=298.0)
    Nstar = _solve_nstar_for_alpha(0.65, T)
    mech = {"RELATIVE_TOLERANCE": 1.0e-10, mech_name: {"reactions": [{"name": "SOAG_to_SOA", "type": "REACTION", "subtype": "SIMPOL_PHASE_TRANSFER", "gas-phase species": "SOAG", "aerosol phase": "mixed", "aerosol-phase species": "SOA", "B": [B1,B2,B3,B4], "N star": Nstar}]}}

    (campdir:=Path(run_dir)/"camp_config").mkdir(parents=True, exist_ok=True)
    (campdir/"aero_phases.json").write_text(json.dumps(phases, indent=2))
    (campdir/"aero_rep.json").write_text(json.dumps(rep, indent=2))
    (campdir/"species.json").write_text(json.dumps(species, indent=2))
    (campdir/"mech.json").write_text(json.dumps(mech, indent=2))
    file_list = {"camp-files": [str((campdir/"aero_phases.json").resolve()), str((campdir/"aero_rep.json").resolve()), str((campdir/"species.json").resolve()), str((campdir/"mech.json").resolve())]}
    (campdir/"config.json").write_text(json.dumps(file_list, indent=2))
    return (campdir/"config.json").resolve()


## Monkey patches (append CAMP to PartMC .spec; write MAM4 camp_link.txt)


In [ ]:
orig_partmc_write = ambrs.partmc.AerosolModel.write_input_files
def patched_partmc_write(self, input, dir: str, prefix: str):
    orig_partmc_write(self, input, dir, prefix)
    if getattr(self, 'camp_config', None):
        cfg = Path(self.camp_config)
        if cfg.is_dir(): cfg = cfg / 'config.json'
        spec_path = Path(dir)/'out'/f"{prefix}.spec"
        txt = spec_path.read_text()
        if 'camp_config' not in txt:
            txt += f"camp_config {cfg.resolve()}\n"
            spec_path.write_text(txt)
ambrs.partmc.AerosolModel.write_input_files = patched_partmc_write

orig_mam4_write = ambrs.mam4.AerosolModel.write_input_files
def patched_mam4_write(self, input, dir: str, prefix: str):
    orig_mam4_write(self, input, dir, prefix)
    if getattr(self, 'camp_config', None):
        cfg = Path(self.camp_config)
        if cfg.is_dir(): cfg = cfg / 'config.json'
        mech = getattr(self, 'camp_mech', None) or 'MAM4_SOA_partitioning'
        (Path(dir)/'camp_link.txt').write_text(f"camp_config={cfg.resolve()}\ncamp_mech={mech}\n")
ambrs.mam4.AerosolModel.write_input_files = patched_mam4_write


## Phase 4a — PartMC-CAMP vs MAM4 (native)


In [ ]:
if RUN_4A:
    run_dir_4a_partmc = PARTMC_DIR/"phase4a"
    run_dir_4a_mam4   = MAM4_DIR/"phase4a"
    for d in [run_dir_4a_partmc, run_dir_4a_mam4]: d.mkdir(parents=True, exist_ok=True)

    partmc_model = ambrs.partmc.AerosolModel(
        processes=ambrs.AerosolProcesses(coagulation=True, condensation=True),
        camp_config=None,
    )
    mam4_model = ambrs.mam4.AerosolModel(
        processes=ambrs.AerosolProcesses(coagulation=True, condensation=True)
    )

    runner = PoolRunner(max_workers=1)
    for sname, sc in zip(scenario_names, scenarios):
        s_part_dir = run_dir_4a_partmc/sname
        s_mam_dir  = run_dir_4a_mam4/sname
        s_part_dir.mkdir(parents=True, exist_ok=True)
        s_mam_dir.mkdir(parents=True, exist_ok=True)

        cfg_path = write_duncan_camp_files(s_part_dir, sc)
        partmc_model.camp_config = str(cfg_path)

        pin = partmc_model.create_input(sc, dt, nsteps)
        partmc_model.write_input_files(pin, str(s_part_dir), sname)

        minp = mam4_model.create_input(sc, dt, nsteps)
        mam4_model.write_input_files(minp, str(s_mam_dir), sname)

        runner.run_one(s_part_dir, PARTMC_EXE, partmc_model.invocation(PARTMC_EXE, sname))
        runner.run_one(s_mam_dir,   MAM4_EXE,   mam4_model.invocation(MAM4_EXE, sname))

    print("Phase 4a complete.")


## Phase 4b — MAM4-CAMP vs MAM4 (native)


In [ ]:
if RUN_4B:
    run_dir_native = MAM4_DIR/"phase4b_native"
    run_dir_camp   = MAM4_DIR/"phase4b_camp"
    for d in [run_dir_native, run_dir_camp]: d.mkdir(parents=True, exist_ok=True)

    mam4_native = ambrs.mam4.AerosolModel(
        processes=ambrs.AerosolProcesses(coagulation=True, condensation=True)
    )
    mam4_camp = ambrs.mam4.AerosolModel(
        processes=ambrs.AerosolProcesses(coagulation=True, condensation=True),
        camp_config=None,
        camp_mech="MAM4_SOA_partitioning",
    )

    runner = PoolRunner(max_workers=1)
    for sname, sc in zip(scenario_names, scenarios):
        s_native = run_dir_native/sname
        s_camp   = run_dir_camp/sname
        s_native.mkdir(parents=True, exist_ok=True)
        s_camp.mkdir(parents=True, exist_ok=True)

        cfg_path = write_duncan_camp_files(s_camp, sc)
        mam4_camp.camp_config = str(cfg_path)

        mi_native = mam4_native.create_input(sc, dt, nsteps)
        mam4_native.write_input_files(mi_native, str(s_native), sname)
        mi_camp = mam4_camp.create_input(sc, dt, nsteps)
        mam4_camp.write_input_files(mi_camp, str(s_camp), sname)

        runner.run_one(s_native, MAM4_EXE, mam4_native.invocation(MAM4_EXE, sname))
        runner.run_one(s_camp,   MAM4_EXE, mam4_camp.invocation(MAM4_EXE, sname))

    print("Phase 4b complete.")


## Phase 4c — PartMC-CAMP vs MAM4-CAMP


In [ ]:
if RUN_4C:
    run_dir_part = PARTMC_DIR/"phase4c"
    run_dir_mam  = MAM4_DIR/"phase4c"
    for d in [run_dir_part, run_dir_mam]: d.mkdir(parents=True, exist_ok=True)

    partmc_model = ambrs.partmc.AerosolModel(
        processes=ambrs.AerosolProcesses(coagulation=True, condensation=True),
        camp_config=None,
    )
    mam4_model = ambrs.mam4.AerosolModel(
        processes=ambrs.AerosolProcesses(coagulation=True, condensation=True),
        camp_config=None,
        camp_mech="MAM4_SOA_partitioning",
    )

    runner = PoolRunner(max_workers=1)
    for sname, sc in zip(scenario_names, scenarios):
        s_part = run_dir_part/sname
        s_mam  = run_dir_mam/sname
        s_part.mkdir(parents=True, exist_ok=True)
        s_mam.mkdir(parents=True, exist_ok=True)

        cfg_p = write_duncan_camp_files(s_part, sc)
        cfg_m = write_duncan_camp_files(s_mam,  sc)
        partmc_model.camp_config = str(cfg_p)
        mam4_model.camp_config   = str(cfg_m)

        pin = partmc_model.create_input(sc, dt, nsteps)
        partmc_model.write_input_files(pin, str(s_part), sname)
        mi  = mam4_model.create_input(sc, dt, nsteps)
        mam4_model.write_input_files(mi, str(s_mam), sname)

        runner.run_one(s_part, PARTMC_EXE, partmc_model.invocation(PARTMC_EXE, sname))
        runner.run_one(s_mam,   MAM4_EXE,   mam4_model.invocation(MAM4_EXE, sname))

    print("Phase 4c complete.")


## Plots (PyParticle-based)


In [ ]:
import matplotlib.pyplot as plt
phase = "4a"  # choose: "4a" | "4c"
if phase == "4a":
    part_dir = PARTMC_DIR/"phase4a"
    mam_dir  = MAM4_DIR/"phase4a"
elif phase == "4c":
    part_dir = PARTMC_DIR/"phase4c"
    mam_dir  = MAM4_DIR/"phase4c"
else:
    part_dir = mam_dir = None

if part_dir and mam_dir:
    fig = plt.figure(figsize=(9,4))
    gs = fig.add_gridspec(1, 2)
    try:
        plot_ensemble_variables(gs=gs, ensemble=None, scenarios=scenario_names,
            timesteps=[30], partmc_dir=str(part_dir), mam4_dir=str(mam_dir),
            variables=["frac_ccn", "bscat"], s_grid=np.logspace(-2,1,30), wavelengths_m=[550e-9])
    except Exception as e:
        print("Plotting error (likely no outputs yet):", e)
    plt.show()
else:
    print("Select phase '4a' or '4c' above.")
